# Prepare robots first  
* run panda repeater on controller pc  

```
ssh panda@192.168.0.172
roslaunch panda_ros_repeater joint_velocity_repeater.launch robot_ip:=192.168.0.13 load_gripper:=false

```

* Keep indy connected to conty to bypass conty-connection delay bug

# initialize notebook

In [1]:
from __future__ import print_function
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:90% !important; } </style>"))
import matplotlib.pyplot as plt

# Initialize constants

In [2]:
from pkg.marker_config import *
from pkg.constraint_graph import *
from pkg.constraint.constraint_action import *
from pkg.constraint.constraint_object import *
from pkg.constants import *
from pkg.utils.plot_utils import *
from pkg.utils.utils import *
from pkg.environment_builder import *
from pkg.ui.ui_broker import *
from pkg.controller.combined_robot import *

gtimer = GlobalTimer.instance()
gtimer.reset()

In [3]:
crob = CombinedRobot(connection_list=(False, False))

connection_list
(False, False)


# initialize graph & ui

In [4]:
if "cam" not in locals():
    cam = None # StereoCamera.instance()

# set urdf
xcustom, JOINT_NAMES, LINK_NAMES, urdf_content = set_custom_robots(crob.robots_on_scene, XYZ_RPY_ROBOTS_DEFAULT, crob.custom_limits, start_rviz=True)
ghnd = GeometryHandle(urdf_content)


graph = ConstraintGraph(ghnd=ghnd, urdf_path=URDF_PATH, joint_names=JOINT_NAMES, link_names=LINK_NAMES, 
                        urdf_content=urdf_content, combined_robot=crob)
graph.set_camera(cam)
graph.set_cam_robot_collision(_add_cam_poles=False)
graph.set_rviz()

# start UI
ui_broker = UIBroker.instance()
ui_broker.initialize(graph)
ui_broker.start_server()

# set rviz
graph.set_rviz(crob.home_pose)
ui_broker.set_tables()

Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran0']/actuator[@name='indy0_motor0']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran1']/actuator[@name='indy0_motor1']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran2']/actuator[@name='indy0_motor2']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran3']/actuator[@name='indy0_motor3']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran4']/actuator[@name='indy0_motor4']
Unknown tag "hardwareInterface" in /robot[@name='custom_robots']/transmission[@name='indy0_tran5']/actuator[@name='indy0_motor5']


Please create a subscriber to the marker
publication OK
published: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Please create a subscriber to the marker
Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "pkg.ui.dash_launcher" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


In [5]:
from pkg.planner.moveit import moveit_py
from pkg.planner.moveit.moveit_planner import MoveitPlanner

### Set Planner

In [6]:
mplan = MoveitPlanner(
    joint_names=graph.joint_names, link_names=graph.link_names, urdf_path=graph.urdf_path, urdf_content=graph.urdf_content, 
    robot_names=crob.robot_names, ghnd=ghnd)

In [7]:
mtem_list=[]

### define constants

In [8]:
GROUP_NAME, TOOL_LINK, BASE_LINK  = "indy0", "indy0_tcp", "base_link"
TOOL_XYZRPY = ((0,0,0.14),(np.pi,0,0))
# TOOL_XYZRPY = ((0,0,0),(0,0,0))
# ORIGIN_XYZRPY = ((0,0,0.4), (0,0,0))
ORIGIN_XYZRPY = (( -0.637836,   -0.102947,  0.527019  ), (3.14151544, -0.01124507,  1.55726101))

TOOL_TF = T_xyzrpy(TOOL_XYZRPY)
TOOL_XYZQUAT = T2xyzquat(TOOL_TF)
ORIGIN_TF = T_xyzrpy(ORIGIN_XYZRPY)

graph.add_marker(ghnd.create_safe(gtype=GEOTYPE.SPHERE, name="tool", link_name="indy0_tcp",
                                      center=TOOL_XYZRPY[0], rpy=(0,0,0), 
                                      dims=(0.02,0.02,0.02), color=(1, 0, 0, 0.7),
                                      collision=False))

# Plane-Cylinder Test

### Show constraints

In [9]:
for mtem in mtem_list:
    graph.remove_geometry(mtem)
mtem_list=[]

In [10]:
mtem_list.append(ghnd.create_safe(gtype=GEOTYPE.BOX, name="plane", link_name="base_link",
                                      center=np.add(ORIGIN_XYZRPY[0], (0,0,0)), rpy=(0,0,0), 
                                      dims=(3,3,0.001), color=(1, 1, 0, 0.5),
                                      collision=False))
mtem_list.append(ghnd.create_safe(gtype=GEOTYPE.CYLINDER, name="cylinder", link_name="base_link",
                                      center=np.add(ORIGIN_XYZRPY[0], (0.05,0,-0.1)), rpy=(np.pi/2,0,0), 
                                      dims=(0.3,0.3,3), color=(1, 1, 0, 0.5),
                                      collision=False))
for mtem in mtem_list:
    graph.add_marker(mtem)

### check starting pose

In [11]:
Q_init = crob.home_pose
start_pose = T2xyzquat(np.matmul(np.matmul(ORIGIN_TF,SE3(Rot_rpy((np.pi,0,0)),[0.0,-0.2,0])), TOOL_TF))
trajectory, success = mplan.planner.plan_py(robot_name=GROUP_NAME, tool_link=TOOL_LINK, goal_pose=start_pose[0]+start_pose[1], goal_link=BASE_LINK, 
                                            Q_init=Q_init, plannerconfig="RRTConnectkConfigDefault", timeout=1)
Q_ready = trajectory[-1]
print("start_pose: {}".format(start_pose))
print("Q_ready: {}".format(np.round(Q_ready,2)))

start_pose: ([-0.8378497958183289, -0.10181383043527603, 0.6669947504997253], [0.7118649008640737, 0.7022939191396269, 0.004029640151390661, -0.003921231635065741])
Q_ready: [-0.47 -0.43 -0.6   0.   -2.11 -0.47  0.   -0.39  0.   -1.57  0.    1.57
  1.57]


In [12]:
graph.show_motion(trajectory)
graph.show_pose(trajectory[-1])

### check goal pose

In [13]:
Q_init = crob.home_pose
# goal_pose = T2xyzquat(np.matmul(np.matmul(ORIGIN_TF,SE3(Rot_rpy((np.pi,0,np.pi/2)),[0,0.05-np.sqrt(0.15**2-0.10**2),0])), TOOL_TF)) # fitst meet
# goal_pose = T2xyzquat(np.matmul(np.matmul(ORIGIN_TF,SE3(Rot_rpy((np.pi,0,np.pi/2)),[0.0,0.05,-0.05])), TOOL_TF)) # top
goal_pose = T2xyzquat(np.matmul(np.matmul(ORIGIN_TF,SE3(Rot_rpy((np.pi,0,np.pi/2)),[0,0.05+0.15,0])), TOOL_TF)) # after meet
trajectory, success = mplan.planner.plan_py(robot_name=GROUP_NAME, tool_link=TOOL_LINK, goal_pose=goal_pose[0]+goal_pose[1], goal_link=BASE_LINK, 
                                            Q_init=Q_ready, plannerconfig="RRTConnectkConfigDefault", timeout=1)
Q_last_ex = trajectory[-1]
print("Q_last_ex: {}".format(Q_last_ex))
print("goal_pose: {}".format(goal_pose))

Q_last_ex: [ 2.61724917e+00 -5.28200320e-02  1.20905390e+00  2.12673962e-03
  1.98386902e+00 -2.09451590e+00  0.00000000e+00 -3.92699082e-01
  0.00000000e+00 -1.57079633e+00  0.00000000e+00  1.57079633e+00
  1.57079633e+00]
goal_pose: ([-0.43788641691207886, -0.10722813755273819, 0.667025625705719], [0.9999612912273312, -0.00676771672940532, 7.665620629702626e-05, -0.005622115354993529])


In [14]:
graph.show_motion(trajectory)
graph.show_pose(trajectory[-1])

### set manifolds

In [18]:
mplan.planner.clear_context_cache()
mplan.planner.clear_manifolds()
ctem_list = make_constraint_list(mtem_list)
mplan.planner.add_union_manifold_py(group_name=GROUP_NAME, tool_link=TOOL_LINK, tool_offset=TOOL_XYZQUAT[0]+TOOL_XYZQUAT[1], 
                                    geometry_list=ctem_list, fix_surface=True, fix_normal=False, tol=2e-3)
ctem_list_vert = make_constraint_list(mtem_list[:1])
mplan.planner.add_union_manifold_py(group_name=GROUP_NAME, tool_link=TOOL_LINK, tool_offset=TOOL_XYZQUAT[0]+TOOL_XYZQUAT[1], 
                                    geometry_list=ctem_list_vert, fix_surface=False, fix_normal=True, tol=2e-3)

### search plan

In [19]:
trajectory, success = mplan.planner.plan_constrained_py(GROUP_NAME,TOOL_LINK, goal_pose[0]+goal_pose[1], BASE_LINK, Q_ready, timeout=60)
print('result: {} ({})'.format(success, len(trajectory)))

result: True (107)


In [20]:
graph.show_motion(trajectory, period=0.01)

## Execute motion

In [70]:
# prepare indy
indy = indytraj_client(server_ip="192.168.0.63", robot_name="NRMK-Indy7")

# Options
traj_type = 1  # 1 for joint move, 2 for task move
traj_freq = 4000  # control rate
dat_size = 6  # DOF

# make_trajectory
trj_exp = trajectory[:,:6]
double_count = int(ceil(log(40000/len(trajectory))/log(2)))
for _ in range(double_count):
    trj_exp = interpolate_double(trj_exp)
trj_exp_d = differentiate(trj_exp, 1./traj_freq)
trj_exp_dd = differentiate(trj_exp_d, 1./traj_freq)
traj_data = np.concatenate([trj_exp, trj_exp_d, trj_exp_dd], axis=-1).flatten()
indy.connect_and(indy.joint_move_to, np.rad2deg(trajectory[0, :6]))
indy.connect_and(indy.move_ext_traj_txt, traj_type, traj_freq, dat_size, traj_data)

Connect: Server IP (192.168.0.63)
Connect: Server IP (192.168.0.63)


### PROCESS
* 2 surface trajectory: checked
* 2 surface-normal jacobian: checked
* 2 surface-normal trajectory: fail due to angle discontinuity, set separate agle constraint
* positive surface: checked
* intersection: checked

In [84]:
# Qtest = trajectory[20]
# Qtest = Q_ready
# Qtest = Q_init
Qtest = Q_last_ex
graph.show_pose(Qtest)
mplan.planner.test_jacobian(moveit_py.JointState(mplan.joint_num, *Qtest))

[-0.0709806,
 -0.0977563,
 -1.05406,
 0.000116293,
 -1.98851,
 1.49946,
 0.0,
 -0.392699,
 0.0,
 -1.5708,
 0.0,
 1.5708,
 1.5708]

# Plane Test

### Show constraints

In [31]:
for mtem in mtem_list:
    graph.remove_geometry(mtem)
mtem_list=[]

In [32]:
mtem_list.append(ghnd.create_safe(gtype=GEOTYPE.BOX, name="plane", link_name="base_link",
                                      center=ORIGIN_XYZRPY[0], rpy=ORIGIN_XYZRPY[1], 
                                      dims=(3,3,0.001), color=(1, 1, 0, 0.5),
                                      collision=False))
for mtem in mtem_list:
    graph.add_marker(mtem)

### check starting pose

In [33]:
Q_init = crob.home_pose
start_pose = T2xyzquat(np.matmul(np.matmul(ORIGIN_TF,SE3(Rot_rpy((0,0,0)),[0]*3)), TOOL_TF))
trajectory, success = mplan.planner.plan_py(robot_name=GROUP_NAME, tool_link=TOOL_LINK, goal_pose=start_pose[0]+start_pose[1], goal_link=BASE_LINK, 
                                            Q_init=Q_init, plannerconfig="RRTConnectkConfigDefault", timeout=1)
Q_ready = trajectory[-1]
print("start_pose: {}".format(start_pose))
print("Q_ready: {}".format(np.round(Q_ready,2)))

start_pose: ([-0.637803852558136, -0.10137300938367844, 0.3870278596878052], [0.003921231635065741, -0.004029640151390661, 0.7022939191396269, 0.7118649008640737])
Q_ready: [ 2.16  2.82 -2.18  0.   -0.64  0.98  0.   -0.39  0.   -1.57  0.    1.57
  1.57]


In [34]:
graph.show_motion(trajectory)
graph.show_pose(trajectory[-1])

### check goal pose

In [35]:
Q_init = crob.home_pose
goal_pose = T2xyzquat(np.matmul(np.matmul(ORIGIN_TF,SE3(Rot_rpy((0,0,0)),[-0.05,0.05,0])), TOOL_TF))
trajectory, success = mplan.planner.plan_py(robot_name=GROUP_NAME, tool_link=TOOL_LINK, goal_pose=goal_pose[0]+goal_pose[1], goal_link=BASE_LINK, 
                                            Q_init=Q_init, plannerconfig="RRTConnectkConfigDefault", timeout=1)
print("goal_pose: {}".format(goal_pose))

goal_pose: ([-0.5884851217269897, -0.15204206109046936, 0.3864694833755493], [0.003921231635065741, -0.004029640151390661, 0.7022939191396269, 0.7118649008640737])


In [37]:
graph.show_motion(trajectory)
graph.show_pose(trajectory[-1])

### set manifolds

In [38]:
mplan.planner.clear_context_cache()
mplan.planner.clear_manifolds()
ctem_list = make_constraint_list(mtem_list)
mplan.planner.add_union_manifold_py(group_name=GROUP_NAME, tool_link=TOOL_LINK, tool_offset=TOOL_XYZQUAT[0]+TOOL_XYZQUAT[1], 
                                    geometry_list=ctem_list, fix_surface=True, fix_normal=True, radius=1e-5, tol=1e-3)

### search plan

In [39]:
tool_tf = np.matmul(get_tf(TOOL_LINK, list2dict(Q_ready, mplan.joint_names), mplan.urdf_content), T_xyzrpy(TOOL_XYZRPY))
goal_pose = T2xyzquat(np.matmul( \
                                np.matmul(ORIGIN_TF, SE3(np.identity(3), (0.0,0.3,0.0))),
                                TOOL_TF))

In [40]:
trajectory, success = mplan.planner.plan_constrained_py(GROUP_NAME,TOOL_LINK, goal_pose[0]+goal_pose[1], BASE_LINK, Q_ready, timeout=60)
print('result: {} ({})'.format(success, len(trajectory)))

result: True (37)


In [42]:
graph.show_motion(trajectory, period=0.01)

# Sphere Test

### Show constraints

In [19]:
for mtem in mtem_list:
    graph.remove_geometry(mtem)
mtem_list=[]

In [20]:
mtem_list.append(ghnd.create_safe(gtype=GEOTYPE.SPHERE, name="cylinder", link_name="base_link",
                                      center=np.add(ORIGIN_XYZRPY[0], (0.05,0,0)), rpy=ORIGIN_XYZRPY[1], 
                                      dims=(0.1,0.1,0.1), color=(1, 1, 0, 0.5),
                                      collision=False))
for mtem in mtem_list:
    graph.add_marker(mtem)

### check starting pose

In [21]:
Q_init = crob.home_pose
start_pose = T2xyzquat(np.matmul(np.matmul(ORIGIN_TF,SE3(Rot_rpy((np.pi/2,0,0)),[0]*3)), TOOL_TF))
trajectory, success = mplan.planner.plan_py(robot_name=GROUP_NAME, tool_link=TOOL_LINK, goal_pose=start_pose[0]+start_pose[1], goal_link=BASE_LINK, 
                                            Q_init=Q_init, plannerconfig="RRTConnectkConfigDefault", timeout=1)
Q_ready = trajectory[-1]
print("start_pose: {}".format(start_pose))
print("Q_ready: {}".format(np.round(Q_ready,2)))

start_pose: ([-0.7778231501579285, -0.10105198621749878, 0.5270082354545593], [0.5061372311797819, 0.4937474126568892, 0.499446172709821, 0.500591766086702])
Q_ready: [-1.07 -0.68 -0.75  1.82 -2.62  0.28  0.   -0.39  0.   -1.57  0.    1.57
  1.57]


In [22]:
graph.show_motion(trajectory)
graph.show_pose(trajectory[-1])

### check goal pose

In [47]:
Q_init = crob.home_pose
goal_pose = T2xyzquat(np.matmul(np.matmul(ORIGIN_TF,SE3(Rot_rpy((np.pi,0,0)),[0,0.05,-0.05])), TOOL_TF))
trajectory, success = mplan.planner.plan_py(robot_name=GROUP_NAME, tool_link=TOOL_LINK, goal_pose=goal_pose[0]+goal_pose[1], goal_link=BASE_LINK, 
                                            Q_init=Q_init, plannerconfig="RRTConnectkConfigDefault", timeout=1)
print("goal_pose: {}".format(goal_pose))

goal_pose: ([-0.5878841280937195, -0.10575991123914719, 0.7170108556747437], [0.7118649008640737, 0.7022939191396269, 0.004029640151390661, -0.003921231635065741])


In [48]:
graph.show_motion(trajectory)
graph.show_pose(trajectory[-1])

### set manifolds

In [49]:
mplan.planner.clear_context_cache()
mplan.planner.clear_manifolds()
ctem_list = make_constraint_list(mtem_list)
mplan.planner.add_union_manifold_py(group_name=GROUP_NAME, tool_link=TOOL_LINK, tool_offset=TOOL_XYZQUAT[0]+TOOL_XYZQUAT[1], 
                                    geometry_list=ctem_list, fix_surface=True, fix_normal=True, radius=1e-5, tol=2e-3)

### search plan

In [50]:
trajectory, success = mplan.planner.plan_constrained_py(GROUP_NAME,TOOL_LINK, goal_pose[0]+goal_pose[1], BASE_LINK, Q_ready, timeout=60)
print('result: {} ({})'.format(success, len(trajectory)))

result: True (100)


In [51]:
graph.show_motion(trajectory, period=0.01)

# Cylinder Test

### Show constraints

In [52]:
for mtem in mtem_list:
    graph.remove_geometry(mtem)
mtem_list=[]

In [53]:
mtem_list.append(ghnd.create_safe(gtype=GEOTYPE.CYLINDER, name="cylinder", link_name="base_link",
                                      center=np.add(ORIGIN_XYZRPY[0], (0.05,0,0)), rpy=(np.pi/2,0,0), 
                                      dims=(0.1,0.1,0.1), color=(1, 1, 0, 0.5),
                                      collision=False))
for mtem in mtem_list:
    graph.add_marker(mtem)

### check starting pose

In [54]:
Q_init = crob.home_pose
start_pose = T2xyzquat(np.matmul(np.matmul(ORIGIN_TF,SE3(Rot_rpy((np.pi/2,0,0)),[0]*3)), TOOL_TF))
trajectory, success = mplan.planner.plan_py(robot_name=GROUP_NAME, tool_link=TOOL_LINK, goal_pose=start_pose[0]+start_pose[1], goal_link=BASE_LINK, 
                                            Q_init=Q_init, plannerconfig="RRTConnectkConfigDefault", timeout=1)
Q_ready = trajectory[-1]
print("start_pose: {}".format(start_pose))
print("Q_ready: {}".format(np.round(Q_ready,2)))

start_pose: ([-0.7778231501579285, -0.10105198621749878, 0.5270082354545593], [0.5061372311797819, 0.4937474126568892, 0.499446172709821, 0.500591766086702])
Q_ready: [-1.07 -0.68 -0.75  1.82 -2.62  0.28  0.   -0.39  0.   -1.57  0.    1.57
  1.57]


In [55]:
graph.show_motion(trajectory)
graph.show_pose(trajectory[-1])

### check goal pose

In [56]:
Q_init = crob.home_pose
goal_pose = T2xyzquat(np.matmul(np.matmul(ORIGIN_TF,SE3(Rot_rpy((np.pi,0,np.pi/2)),[0.0,0.05,-0.05])), TOOL_TF))
trajectory, success = mplan.planner.plan_py(robot_name=GROUP_NAME, tool_link=TOOL_LINK, goal_pose=goal_pose[0]+goal_pose[1], goal_link=BASE_LINK, 
                                            Q_init=Q_ready, plannerconfig="RRTConnectkConfigDefault", timeout=1)
print("goal_pose: {}".format(goal_pose))

goal_pose: ([-0.5878841280937195, -0.10575991123914719, 0.7170108556747437], [0.9999612912273312, -0.00676771672940532, 7.665620629702626e-05, -0.005622115354993529])


In [57]:
graph.show_motion(trajectory)
graph.show_pose(trajectory[-1])

### set manifolds

In [58]:
mplan.planner.clear_context_cache()
mplan.planner.clear_manifolds()
ctem_list = make_constraint_list(mtem_list)
mplan.planner.add_union_manifold_py(group_name=GROUP_NAME, tool_link=TOOL_LINK, tool_offset=TOOL_XYZQUAT[0]+TOOL_XYZQUAT[1], 
                                    geometry_list=ctem_list, fix_surface=True, fix_normal=True, radius=1e-5, tol=2e-3)

### search plan

In [59]:
trajectory, success = mplan.planner.plan_constrained_py(GROUP_NAME,TOOL_LINK, goal_pose[0]+goal_pose[1], BASE_LINK, Q_ready, timeout=60)
print('result: {} ({})'.format(success, len(trajectory)))

result: True (133)


In [61]:
graph.show_motion(trajectory, period=0.01)